In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import py3Dmol

from proteome import protein
from proteome.models.design.protein_seq_des import config
from proteome.models.design.protein_seq_des.modeling import ProteinSeqDesForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

PyRosetta-4 2023 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python310.Release 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [3]:
designer = ProteinSeqDesForSequenceDesign("conditional_model_0", random_seed=37)
folder = OmegaFoldForFolding()

In [4]:
with open("tim10.pdb", mode="r") as f:
    gt_pdb = f.read()

In [5]:
target_protein = protein.from_pdb_string(gt_pdb)

In [6]:
sequence, energy = designer.design_sequence(
    target_protein, 
    inference_config=config.InferenceConfig(sampler_config=config.SamplerConfig(is_tim=True)),
)
print(f"Sequence: {sequence} with energy {energy:.3f}")

Running design: 100%|███████████████████████████████████████████| 49/49 [01:13<00:00,  1.49s/it]

Sequence: RVAILDLEDTDEMIDDVKKLSSAGCPKIGTQTGLVEEALTAVFAGSYVVYVKGEDTEDAVRLVRRFAAKGYELIAVLSKSEEEMAKGLAAGAVICVIELDDAAESIQDVQKLAKFGAPLIMAIAKSSEELQGAAAAGALLLIINPADVADSIKEVDELAAAGAKIIATLATDAKRLHNAAEAG with energy 8444.171


In [7]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [8]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol